In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import numpy as np

In [3]:
data = pd.read_csv("supply_chain_data.csv")
data.head()

,Product type,SKU,Price,Availability,Number of products sold,Revenue generated,Customer demographics,Stock levels,Lead times,Order quantities,...,Location,Lead time,Production volumes,Manufacturing lead time,Manufacturing costs,Inspection results,Defect rates,Transportation modes,Routes,Costs
0,haircare,SKU0,69.808006,55,802,8661.996792,Non-binary,58,7,96,...,Mumbai,29,215,29,46.279879,Pending,0.226410,Road,Route B,187.752075
1,skincare,SKU1,14.843523,95,736,7460.900065,Female,53,30,37,...,Mumbai,23,517,30,33.616769,Pending,4.854068,Road,Route B,503.065579
2,haircare,SKU2,11.319683,34,8,9577.749626,Unknown,1,10,88,...,Mumbai,12,971,27,30.688019,Pending,4.580593,Air,Route C,141.920282
3,skincare,SKU3,61.163343,68,83,7766.836426,Non-binary,23,13,59,...,Kolkata,24,937,18,35.624741,Fail,4.746649,Rail,Route A,254.776159
4,skincare,SKU4,4.805496,26,871,2686.505152,Non-binary,5,3,56,...,Delhi,5,414,3,92.065161,Fail,3.145580,Air,Route A,923.440632


In [4]:
target_column = "Manufacturing costs"
feature_column = "Production volumes"

In [5]:
x = data[feature_column].values.reshape(-1,1)
y = data[target_column].values

In [6]:
scaler = MinMaxScaler()
x_scaled = scaler.fit(x)

In [7]:
x_train, x_test, y_train, y_test = train_test_split(data[feature_column], data[target_column], 
                                                    test_size=0.2, random_state=42)

In [8]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64,activation='relu',input_dim=1),
    tf.keras.layers.Dense(32,activation='relu'),
    tf.keras.layers.Dense(1)
])

In [9]:
model.compile(optimizer='adam',loss='mean_square_error')

In [12]:
print("x shape:", x.shape)
print("y shape:", y.shape)

x shape: (100, 1)
y shape: (100,)


In [13]:
min_production_volume = data["Order quantities"].min()
max_production_volume = 1000
step_size = 10

In [14]:
cheapest_cost = float("inf")
best_production_volume = None

In [16]:
for production_volume in range(min_production_volume,max_production_volume + 1, step_size):
    normalized_production_volume = scaler.transform(np.array([[production_volume]]))
    predicted_cost = model.predict(normalized_production_volume)
    if production_volume == best_production_volume:
        best_cost = predicted_cost[0][0]
    if predicted_cost[0][0] >= 0:
        cheapest_cost = predicted_cost[0][0]
        best_production_volume = production_volume
print("The Most Optimal Production Volume to Minimize Manufacturing Cost:", best_production_volume)
print("The Cheapest Manufacturing Cost", cheapest_cost)

1/1 [==============================] - 1s 1s/step
The Most Optimal Production Volume to Minimize Manufacturing Cost: 101
The Cheapest Manufacturing Cost 0.00018357573
